# Tableau Homework - Citi Bike Analytics

Since 2013, the Citi Bike Program has implemented a robust infrastructure for collecting data on the program's utilization. Through the team's efforts, each month bike data is collected, organized, and made public on the Citi Bike Data [webpage](https://www.citibikenyc.com/system-data)

Task
Your task in this assignment is to aggregate the data found in the Citi Bike Trip History Logs and find two unexpected phenomena.

Design 2-5 visualizations for each discovered phenomena (4-10 total). You may work with a timespan of your choosing. Optionally, you may merge multiple datasets from different periods.

### Import Dependencies

In [1]:
import os
import pandas as pd

### Import and clean the data

In [2]:
directory = os.path.join("data","NY-2018-2020")

In [3]:
# Get list of files on the directory
files_list = [];
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        files_list.append(os.path.join(filename))

    else:
        continue
print(files_list)

['202001-citibike-tripdata.csv', '202002-citibike-tripdata.csv', '202003-citibike-tripdata.csv', '202004-citibike-tripdata.csv', '202005-citibike-tripdata.csv', '202006-citibike-tripdata.csv', '202007-citibike-tripdata.csv', '202008-citibike-tripdata.csv', '202009-citibike-tripdata.csv', '202010-citibike-tripdata.csv', '202011-citibike-tripdata.csv']


In [4]:
df_merged = pd.DataFrame({'A' : []})

In [ ]:
n = 0
for file in files_list:
    print('--'*30)
    print(f"File {n} of {len(files_list)}: {file}")
    if n == 0:
        filepath = os.path.join(directory,file)
        df_merged = pd.read_csv(filepath)
    else:
        filepath = os.path.join(directory,file)
        df = pd.read_csv(filepath)
        df_merged = pd.merge(df, df_merged, how='outer')
    
    n += 1

______________________________
202001-citibike-tripdata.csv
______________________________
202002-citibike-tripdata.csv
______________________________
202003-citibike-tripdata.csv
______________________________
202004-citibike-tripdata.csv
______________________________
202005-citibike-tripdata.csv
______________________________
202006-citibike-tripdata.csv
______________________________
202007-citibike-tripdata.csv
______________________________
202008-citibike-tripdata.csv
______________________________
202009-citibike-tripdata.csv
______________________________
202010-citibike-tripdata.csv


In [ ]:
df_merged

In [ ]:
df_merged.isnull().sum()

In [ ]:
df_merged.isnull().values.any()

In [ ]:
# creating bool series True for NaN values  
bool_series = pd.isnull(df_merged['starttime'])  
    
bool_series.describe()

### Analytics

* How many trips have been recorded total during the chosen period?

In [ ]:
# How many trips have been recorded total during the chosen period?
total_trips = df_merged['tripduration'].count()
print(f"How many trips have been recorded total during the chosen period? \nR:{total_trips}")

<hr style="border-top: 3px solid gray;">

* By what percentage has total ridership grown?

In [ ]:
df_merged.index = pd.to_datetime(df_merged['starttime'],format='%Y-%m-%d')
df_grouped_starttime = df_merged.groupby(by=[ df_merged.index.year])
# df_grouped_starttime = df_merged.groupby(by=[ df_merged.index.year, df_merged.index.month])

In [ ]:
df_grouped_year = df_grouped_starttime.count()
df_grouped_year

In [ ]:
ax = df_grouped_year['tripduration'].plot(kind="bar",color="black", alpha=0.5,rot=0,figsize=(15, 5))
ax.set_xlabel("");
ax.set_ylabel("Trips");
ax.set_title("Total Bike Trips by Year");

In [ ]:
growth_2019 = df_grouped_year['tripduration'].iloc[1] / df_grouped_year['tripduration'].iloc[0]
growth_2020 = df_grouped_year['tripduration'].iloc[2] / df_grouped_year['tripduration'].iloc[1]

In [ ]:
print(f"Growth 2019: {round(growth_2019*100)}%")
print(f"Growth 2020*: {round(growth_2020*100)}%")
print("\n* for 2020 still missing December")

<hr style="border-top: 3px solid gray;">

* How has the proportion of short-term customers and annual subscribers changed?

In [ ]:
df_Subscriber = df_merged.loc[df_merged['usertype'] == 'Subscriber']
df_Subscriber_grouped_year = df_Subscriber.groupby(by=[ df_Subscriber.index.year]).count()
df_Subscriber_grouped_year

In [ ]:
subscriber_to_total = 100*df_Subscriber_grouped_year['tripduration']/df_grouped_year['tripduration']
subscriber_to_total.plot(kind='bar',figsize=(15, 5))

In [ ]:
customer_to_total = 100*(1-df_Subscriber_grouped_year['tripduration']/df_grouped_year['tripduration'])
ax = customer_to_total.plot(kind='bar',color="r", alpha=0.5,rot=0,figsize=(15, 5))
ax.set_xlabel("");
ax.set_ylabel("%");
ax.set_title("Non-Subscriber's Growth by Year");

<hr style="border-top: 3px solid gray;">

* What are the peak hours in which bikes are used during summer months?

In [ ]:
# summer (June, July, August)
df_resampled_year = df_merged['starttime'].resample('Y')
df_resampled_year


In [ ]:
df_resampled_year.count()

In [ ]:
df_resampled_by_hour = df_merged['starttime'].resample('H').count()

In [ ]:
df_resampled_by_hour

In [ ]:
df_resampled_by_hour_summer_2018 = df_resampled_by_hour.loc[(df_resampled_by_hour.index >= "2018-06-01")]

In [ ]:
df_resampled_by_hour_summer_2018 = df_resampled_by_hour_summer_2018.loc[(df_resampled_by_hour_summer_2018.index < "2018-09-01")]

In [ ]:
df_summer_2018 = pd.DataFrame(df_resampled_by_hour_summer_2018.resample('H').sum())

In [ ]:
df_summer_2018.plot()

In [ ]:
ax = df_summer_2018.groupby(by=[ df_summer_2018.index.hour]).sum().plot(kind="bar", color="g", alpha=0.5,rot=0,figsize=(15, 5))
ax.set_xlabel("Hour");
ax.set_ylabel("Trips");
ax.set_title("Trips by Hour (Summer 2018)");

In [ ]:
df_resampled_by_hour_summer_2019 = df_resampled_by_hour.loc[(df_resampled_by_hour.index >= "2019-06-01")]
df_resampled_by_hour_summer_2019 = df_resampled_by_hour_summer_2019.loc[(df_resampled_by_hour_summer_2019.index < "2019-09-01")]
df_summer_2019 = pd.DataFrame(df_resampled_by_hour_summer_2019.resample('H').sum())
ax = df_summer_2019.groupby(by=[ df_summer_2019.index.hour]).sum().plot(kind="bar", color="b", alpha=0.5,rot=0,figsize=(15, 5))
ax.set_xlabel("Hour");
ax.set_ylabel("Trips");
ax.set_title("Trips by Hour (Summer 2019)");

In [ ]:
df_resampled_by_hour_summer_2020 = df_resampled_by_hour.loc[(df_resampled_by_hour.index >= "2020-06-01")]
df_resampled_by_hour_summer_2020 = df_resampled_by_hour_summer_2020.loc[(df_resampled_by_hour_summer_2020.index < "2020-09-01")]
df_summer_2020 = pd.DataFrame(df_resampled_by_hour_summer_2020.resample('H').sum())
ax = df_summer_2020.groupby(by=[ df_summer_2020.index.hour]).sum().plot(kind="bar", color="r", alpha=0.5,rot=0,figsize=(15, 5))
ax.set_xlabel("Hour");
ax.set_ylabel("Trips");
ax.set_title("Trips by Hour (Summer 2020)");

<hr style="border-top: 3px solid gray;">

* What are the peak hours in which bikes are used during winter months?

In [ ]:
df_resampled_by_hour_winter_2018 = df_resampled_by_hour.loc[(df_resampled_by_hour.index >= "2018-01-01")]
df_resampled_by_hour_winter_2018 = df_resampled_by_hour_winter_2018.loc[(df_resampled_by_hour_winter_2018.index < "2018-03-01")]
df_winter_2018 = pd.DataFrame(df_resampled_by_hour_winter_2018.resample('H').sum())
ax = df_winter_2018.groupby(by=[ df_winter_2018.index.hour]).sum().plot(kind="bar", color="g", alpha=0.5,rot=0,figsize=(15, 5))
ax.set_xlabel("Hour");
ax.set_ylabel("Trips");
ax.set_title("Trips by Hour (Winter 2018)");

In [ ]:
df_resampled_by_hour_winter_2019 = df_resampled_by_hour.loc[(df_resampled_by_hour.index >= "2019-01-01")]
df_resampled_by_hour_winter_2019 = df_resampled_by_hour_winter_2019.loc[(df_resampled_by_hour_winter_2019.index < "2019-03-01")]
df_winter_2019 = pd.DataFrame(df_resampled_by_hour_winter_2019.resample('H').sum())
ax = df_winter_2019.groupby(by=[ df_winter_2019.index.hour]).sum().plot(kind="bar", color="b", alpha=0.5,rot=0,figsize=(15, 5))
ax.set_xlabel("Hour");
ax.set_ylabel("Trips");
ax.set_title("Trips by Hour (Winter 2019)");

In [ ]:
df_resampled_by_hour_winter_2020 = df_resampled_by_hour.loc[(df_resampled_by_hour.index >= "2020-01-01")]
df_resampled_by_hour_winter_2020 = df_resampled_by_hour_winter_2020.loc[(df_resampled_by_hour_winter_2020.index < "2020-03-01")]
df_winter_2020 = pd.DataFrame(df_resampled_by_hour_winter_2020.resample('H').sum())
ax = df_winter_2020.groupby(by=[ df_winter_2020.index.hour]).sum().plot(kind="bar", color="r", alpha=0.5,rot=0,figsize=(15, 5))
ax.set_xlabel("Hour");
ax.set_ylabel("Trips");
ax.set_title("Trips by Hour (Winter 2020)");

<hr style="border-top: 3px solid gray;">

* Today, what are the top 10 stations in the city for starting a journey? (Based on data, why do you hypothesize these are the top locations?)

In [ ]:
top10_start_stations = df_merged.loc[(df_merged.index >= "2020-11-01")].groupby("start station name").count().sort_values("tripduration",ascending=False).head(10)
ax = top10_start_stations['tripduration'].plot(kind="bar", color="fuchsia", alpha=1,figsize=(15, 5))
ax.set_xlabel("");
ax.set_ylabel("Trips");
ax.set_title("Top 10 Stations For Starting A Journey");

<hr style="border-top: 3px solid gray;">

* Today, what are the top 10 stations in the city for ending a journey? (Based on data, why?)

In [ ]:
top10_ending_stations = df_merged.loc[(df_merged.index >= "2020-11-01")].groupby("end station name").count().sort_values("tripduration",ascending=False).head(10)
ax = top10_ending_stations['tripduration'].plot(kind="bar", color="tan", alpha=1,figsize=(15, 5))
ax.set_xlabel("");
ax.set_ylabel("Trips");
ax.set_title("Top 10 Stations For Ending A Journey");

<hr style="border-top: 3px solid gray;">

* Today, what are the bottom 10 stations in the city for starting a journey? (Based on data, why?)

In [ ]:
bottom10_start_stations = df_merged.loc[(df_merged.index >= "2020-11-01")].groupby("start station name").count().sort_values("tripduration",ascending=True).head(10).sort_values("tripduration",ascending=False)
ax = bottom10_start_stations['tripduration'].plot(kind="bar", color="sienna", alpha=1,figsize=(15, 5))
ax.set_xlabel("");
ax.set_ylabel("Trips");
ax.set_title("Bottom 10 Stations For Starting A Journey");

<hr style="border-top: 3px solid gray;">

* Today, what are the bottom 10 stations in the city for ending a journey (Based on data, why?)

In [ ]:
bottom10_ending_stations = df_merged.loc[(df_merged.index >= "2020-11-01")].groupby("end station name").count().sort_values("tripduration",ascending=True).head(10).sort_values("tripduration",ascending=False)
ax = bottom10_ending_stations['tripduration'].plot(kind="bar", color="teal", alpha=1,figsize=(15, 5))
ax.set_xlabel("");
ax.set_ylabel("Trips");
ax.set_title("Bottom 10 Stations For Ending A Journey");

<hr style="border-top: 3px solid gray;">

* Today, what is the gender breakdown of active participants (Male v. Female)?

In [ ]:
gender_breakdown = df_merged.loc[(df_merged.index >= "2020-11-01") & (df_merged.gender != 0)].groupby("gender")
ax = gender_breakdown['tripduration'].count().plot(kind="bar", rot=0,color="violet", alpha=1,figsize=(15, 5))
ax.set_xlabel("1=male; 2=female");
ax.set_ylabel("Trips");
ax.set_title("Gender Breakdown");

<hr style="border-top: 3px solid gray;">

* How effective has gender outreach been in increasing female ridership over the timespan?

In [ ]:
female_ridership_by_month = df_merged['tripduration'].loc[(df_merged.gender == 2)].resample("M").count() / df_merged['tripduration'].resample("M").count()
ax = female_ridership_by_month.plot(color="violet",marker='*', alpha=1,figsize=(15, 5))
ax.set_xlabel("");
ax.set_ylabel("Trips");
ax.set_title("Female Ridership By Month");

<hr style="border-top: 3px solid gray;">

* How does the average trip duration change by age?

In [ ]:
df_merged['birth year'].value_counts()

In [ ]:
ax = df_merged['birth year'].hist(bins=50,color="b", alpha=.3,figsize=(15, 5))
ax.set_xlabel("Birth Year");
ax.set_ylabel("Trip Frequency");
ax.set_title("Birth Year Distribution By Trip");

In [ ]:
df_grouped_age = df_merged.groupby("birth year")

In [ ]:
ax = df_grouped_age['tripduration'].mean().plot(kind="bar",color="k", alpha=1,figsize=(18, 5))
ax.set_xlabel("Year");
ax.set_ylabel("Average Trip Duration (s)");
ax.set_title("Average Trip Duration By Birth Date");

In [ ]:
# Show the outlier for Birth Year 1902
df_merged.loc[(df_merged['birth year'] == 1902)]

In [ ]:
# Show the outlier for Birth Year 1910
df_merged.loc[(df_merged['birth year'] == 1910) & (df_merged['tripduration'] > 5000)]

In [ ]:
df_grouped_age_usertype = df_merged.groupby(["birth year","usertype"]).count()

In [ ]:
df_grouped_age_usertype

In [ ]:
# Verifind the outlier for birth year 1969
df_merged['usertype'][df_merged['birth year'] == 1969].value_counts()

In [ ]:
df_merged[(df_merged['birth year'] == 1969) & (df_merged['usertype'] == 'Customer')]

In [ ]:
df_merged[['starttime','tripduration']][(df_merged['birth year'] == 1969) & (df_merged['usertype'] == 'Customer')].groupby('starttime').sum()

<hr style="border-top: 3px solid gray;">

* What is the average distance in miles that a bike is ridden?

In [ ]:
from geopy.distance import geodesic

origin = (df_merged['start station latitude'].iloc[0], df_merged['start station longitude'].iloc[0])
dist = (df_merged['end station latitude'].iloc[0], df_merged['end station longitude'].iloc[0])
print(f"Distance is {round(100*geodesic(origin, dist).miles)/100} miles")

In [ ]:
list_distances = []
for index in range(len(df_merged.index)):
    origin = (df_merged['start station latitude'].iloc[index], df_merged['start station longitude'].iloc[index])
    dist = (df_merged['end station latitude'].iloc[index], df_merged['end station longitude'].iloc[index])
#     print(f"Distance is {round(100*geodesic(origin, dist).miles)/100} miles")
    list_distances.append(round(100*geodesic(origin, dist).miles)/100)

In [ ]:
average_distance = sum(list_distances)/len(list_distances)

In [ ]:
average_distance

In [ ]:
pd.DataFrame(list_distances).hist()

<hr style="border-top: 3px solid gray;">

* Which bikes (by ID) are most likely due for repair or inspection in the timespan?

In [ ]:
df_groupby = df_merged.groupby("bikeid")

In [ ]:
ax = df_groupby['tripduration'].sum().sort_values(ascending=False).head(15).plot(kind="bar", color="y", alpha=1,figsize=(15, 5))
ax.set_xlabel("bikeid");
ax.set_ylabel("Trip Duration (s)");
ax.set_title("Top 10 Bikes By Total Trip Time");

In [ ]:
ax = df_groupby['tripduration'].sum().sort_values(ascending=True).head(15).sort_values(ascending=False).plot(kind="bar", color="g", alpha=1,figsize=(15, 5))
ax.set_xlabel("bikeid");
ax.set_ylabel("Trip Duration (s)");
ax.set_title("Bottom 10 Bikes By Total Trip Time");

<hr style="border-top: 3px solid gray;">

* How variable is the utilization by bike ID?

In [ ]:
ax = df_groupby['tripduration'].count().plot(color="k", alpha=1,figsize=(15, 5))
ax.set_xlabel("bikeid");
ax.set_ylabel("Trip");
ax.set_title("Total Trips by Bike");

In [ ]:
ax = df_groupby['tripduration'].sum().plot(color="r", alpha=1,figsize=(15, 5))
ax.set_xlabel("bikeid");
ax.set_ylabel("Trip Duration (s)");
ax.set_title("Total Trip Duration by Bike");

<hr style="border-top: 3px solid gray;">